# Extracting League of Legends Match Data using Riot API - PART I, II and III

## Imports

In [1]:
import requests
import pandas as pd
import time

Put your Riot API key here

In [2]:
api_key = 'RGAPI-2174184f-c778-40c0-8891-56104e41f1fb'

## Part I: [League - V4 Endpoint](https://developer.riotgames.com/apis#league-v4/GET_getMasterLeague) 

Get summoner information from master + players

Testing how to do it with the Challenger League - V4 ENDPOINT

In [7]:
api_ulr = 'https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Challenger ENDPOINT
 
resp = requests.get(api_ulr) # Sending the get request to the url
challenger_summoners = pd.json_normalize(resp.json()['entries']) # Use json_normalize to the JSON data returned by the get request to turn it into a flat table.
challenger_summoners.head()

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,fo9vhM_MeAnq2UByTbKMYgMALReUXtYI-OYhLXUupd0a,90kg fibrado,995,I,709,666,False,False,True,False
1,HPdsxp8FNV4F8zx74UjBDkvWXwEPGZp3ed6z8s68Y6lem-Q,thick b4rk,1501,I,199,130,True,False,False,False
2,Ww-CgMlK2M1Hfdv4AIaE-c3KJMSZI80iEZN9F3B5YJdd0jw,TOP PROMISE,956,I,275,235,False,False,True,False
3,AE3vmXe7Ym8zwCvmAGlFbRmPvAze5q4Pk2BGZIdAgSyra94,Leving,1123,I,199,156,True,False,False,False
4,4GngGhCiOkc17VFUu_7Wg7VQAY90OIo07_7DVevp5jVrczU,RISE nZ MIDKING,986,I,488,443,True,False,False,False


Creating a function to get all the information of players from Master to Challanger in Brazil, using the [League V4 Endpoint](https://developer.riotgames.com/apis#league-v4/GET_getMasterLeague). 

In [3]:
def get_summonerId(api_key):
    challenger_url = 'https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Challanger ENDPOINT
    grandmaster_url = 'https://br1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Grandmaster EDNPOINT
    master_url = 'https://br1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={}'.format(api_key) # Master ENDPOINT

    league_url_lists = [challenger_url, grandmaster_url, master_url] # Creating a list with the 3 endpoints (master, grandmaster and challenger)

    df_list = [] # A list that will store the response from the get request (league_df)

    for url in league_url_lists: # Loop interation through league_url_lists
        resp = requests.get(url) # Sending a get request to the url
        league_df = pd.json_normalize(resp.json()['entries'])
        df_list.append(league_df)
    
    final_league_df = pd.concat(df_list) # Concatenating the stored information in league_url_lists to create the final_league_df

    return final_league_df

In [4]:
# Executing the function
league_df = get_summonerId(api_key)
league_df.head()

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,12U30x4l2wRRf0j-fUPOKe4S_X8ltJVTy9GUmdwig3VJS8I,Little Mandiocaa,1106,I,477,424,True,False,False,False
1,luphOgN2p4DzpsK4R9cXCkYw5wVTPuJnRy2J0fdAsMdGCVM,Tutsz,1623,I,512,429,True,False,False,False
2,Urv1dLL_5vIT3p43U-OMyyCcFuos4JqWRvhVHhVeRJV_qQ,haxq,1014,I,251,208,True,False,False,False
3,Ee43S-uA6mcG2Oglx6hQgAaOi7S0eHNGUn3OYHKK-GIPO2c,Drakehero,1298,I,554,488,True,False,False,False
4,AAeJEWWyt0UXmDvMZJMytGCUpQ1MrYOESnPC_FY0eNtff_...,GEN 1EHENDS,1743,I,182,145,True,False,False,False


## PART II: 

Creating a fucntion to retrieve the puuid and other information for each `summonerId` in the league_df, using the [summoner V4 Endpoint](https://developer.riotgames.com/apis#summoner-v4/GET_getBySummonerId)

In [5]:
def get_summonerinfo_by_summonerId(df, api_key):
    
    summoner_list = []

    for i in range(len(df)):

        # Creating a dynamic api_ulr for each register in the summonerId column
        api_url = ('https://br1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)

        resp = requests.get(api_url) # Sending a get request to the api_url
        print(resp.status_code)

        if resp.status_code == 200:
            pass
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 10 second') #approximate 110 second wait
                    time.sleep(10)

                    api_url = ('https://br1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}'.format(df['summonerId'].iloc[i],api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break

        summoner_info = pd.json_normalize(resp.json()) # Use json_normalize to the JSON data returned by the get request to turn it into a flat table.
        summoner_list.append(summoner_info) # Appending the DataFrame generated to the list "summoner_list"
        
    final_summoner_info = pd.concat(summoner_list) # Using pd.concat to concatenate the results stored in the "summoner_list"
            
    return final_summoner_info
    

In [ ]:
# Executing the function
summoner_df = get_summonerinfo_by_summonerId(league_df, api_key)

In [ ]:
summoner_df.reset_index(inplace = True) # Reseting the Index
summoner_df.drop(columns='index', inplace = True) # Droping the column Index
summoner_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,fN3HwF-H2cjFqKTFKzVwdI_Dti6s1-F0VItvl_HJIKU7Yw,25hQJKGx7U9zvSKEW9Oy61MXOF-_JUB0yYRcregXmh_0jxA,907TX3ji-JNb3NFS2m0hq0SBc4t0MFIuR0zG_nN-MAe1KY...,DIMI CHEERLEADER,3152,1665373919000,574
1,cpuvfpOwz3hwkA3eLvYg3opyVK9HvCSeRto5b3lAypC9Lg,K_t1CG_0G2PDtTYbAQsuM7VOKig08YP3Nuth6_mHT7RtkOg,ycOnsXgi9qaISMpIthPzqHNgzcMm-06EnjR_bRMskxCNYZ...,seixon o tal,4763,1664925545000,489
2,ur4ztwGXXFIKNvfwgnAR91kPooxHOMNQF9YD1I9Ogmi7Qa0,cNeIdFADyYa6814BcxY5dZChU62EU4H6Rxhvv2KRWxnoAeY,VML4iXC3kfiKfsdvScvhIIbozakIIWHtPycuDuFJKY0FR_...,Tar2an,8,1664989875125,303
3,2cwshbyA2Q25Qt9L27_HtkIp9z8gNL1yKCPL2Pawdw_NMME,CYqPrEdAzT4wNDD1jFpaLdb_EppHvBQ4zUVJtqBQsTRJE1...,e25agtLewVaTlQJELhyu2Dam6Mhxm_HVzPzz-AeQdk57yB...,TuruPaoComTrigo,4626,1665557458000,631
4,MNl05HbbjRB-LIa329JiNrGw5ZRXP19t-RBy3galp6SfDeA,G37kyfuAyUjSZhpOthLUoU2wUUQgbr5OFOHmkQGnrMors6...,Cogvp72JnclOnoSc_f1h1MlqeNxNC08fOJRzt0f1H4RM-I...,paiN brTT RXRXRX,5026,1665154930000,258
...,...,...,...,...,...,...,...
4394,jBQNo23RJhx9x1dui5wqwNrhWd3XnEyAZwon6O9qoMsDS0A,pHDRYXGy7LSxGBQrWb8bHxmDkkq1TFASoNLPGjKV8Ft53QA,yIZ39lJb9gDDv9h2MiBL8Yt5ZN4QakAgDyH9H_avuSFgyw...,paiN anato,5503,1665157376000,393
4395,G2npnup_ZMCmKpvjblO2LN8oBBoI7jQG71KUPWyrc0z9XA,ZMtFN7dLQGH25Cje-e7LIfvG1-YspeCulUhyNZHIfd6OY5Y,Ncu19ZIg26Wo41Vo_Lz19sx0_Rzyt7rRXafDTQRjWF3-IT...,Aluminnium,5528,1665101684000,472
4396,y_rilf706xhftDmUfvmu0A5VJ6IevYttkUfavhugBlZB0DE,adN-UM1Wacj9PBJZuGHA787pmo1vSnpV1XF2c__zx9yVnkU,yconNZaHz8VYdIuJmZratxbC8nRB_ZhU4RQADAscPHhCSK...,ssnow,3855,1665254322000,462
4397,Bamm_GZmtjAHC0ct9LASI5ywKMe7v-lRyY2-u4jn3jkbXBc,OtTJkX4X7AcbgwsMNPv8tUFq2t3_8NMnL_mc5l_ib2SoOR4,rxRcyC5TKROKt_sPYD6pzgYT6fA9UzGlCrxDLpy0je0rHI...,BT1 Ryuho,5521,1665365396000,653


Since the code takes about 1:30 hour to execute is important to save the results for further usage.

In [ ]:
summoner_df.to_csv('summoner_df.csv', index = False) 

## PART III:

Creating a fucntion to retrieve the last 20 matchids for each `puiid` in the summoner_df, using the [match V5 Endpoint](https://developer.riotgames.com/apis#match-v5/GET_getMatch)

In [15]:
def get_matchids_by_puiid(df, api_key):

    matchids_list = []

    for i in range(len(df)):

        api_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?type=ranked&start=0&count=20&api_key={}'.format(df['puuid'].iloc[i], api_key)

        resp = requests.get(api_url) # Sending a get request to the api_url
        print(resp.status_code)

        if resp.status_code == 200:
            pass
        elif resp.status_code == 429:
            while True: # while loop because of riot api cost
                if resp.status_code == 429: #429 error is api cost issue
                    print('429 delay try 100 second') #approximate 110 second wait
                    time.sleep(10)

                    api_url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?type=ranked&start=0&count=20&api_key={}'.format(df['puuid'].iloc[i], api_key)
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break        

        matchids = pd.DataFrame(resp.json(), columns = ['matchid']) # Use json_normalize to the JSON data returned by the get request to turn it into a flat table.
        matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"
    
    matchids_df = pd.concat(matchids_list) # Using pd.concat to concatenate the results stored in the "matchids_list"
    
    return matchids_df

In [18]:
# Executing the function
matchids_df = get_matchids_by_puiid(summoner_df, api_key)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
429
429 delay try 100 second
429
429 delay try 100 second
429
429 delay try 100 second
200
limit cost resolve
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
429
429 delay try 100 second
429
429 delay try 100 second
429
429 delay try 100 second
200
lim

In [ ]:
# Reseting Index
matchids_df.reset_index(inplace=True)
matchids_df.drop(columns = 'index', inplace = True )
matchids_df.shape()

,matchid
0,BR1_2605729365
1,BR1_2605701933
2,BR1_2605689398
3,BR1_2605400824
4,BR1_2604585739
5,BR1_2604095861
6,BR1_2604082590
7,BR1_2604046644
8,BR1_2603425361
9,BR1_2603345504


Just in case I will drop duplicates out of matchids_df, since we only want uniqued matchids


In [26]:
# Dropping duplicates
matchids_df_without_duplicates = matchids_df.drop_duplicates()

In [30]:
matchids_count_before = matchids_df.shape[0]
matchids_count_after = matchids_df_without_duplicates.shape[0]
print('Before we had {} matchids, after we have {} matchids'.format(matchids_count_before, matchids_count_after))

Before we had 87980 matchids, after we have 37971 matchids


In [29]:
# Saving the File to use in the next notebook
matchids_df_without_duplicates.to_csv('matchids_df.csv', index = False)